### aim: 

### date: 

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


: 

In [ ]:
%reset

: 

In [ ]:
import numpy as np
from cmocean import cm
import cartopy as cp
import cartopy.crs as ccrs
import netCDF4 as nc
import matplotlib.pyplot as plt
import xarray as xr
import sys
sys.path.append('/gpfs/home/mep22dku/scratch/SOZONE')
#list of models
sys.path.append('/gpfs/home/mep22dku/scratch/SOZONE/UTILS')
import lom
import utils as ut

%matplotlib inline
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')
import cartopy.feature as cfeature
from importlib import reload
import matplotlib.path as mpath
import glob
import pickle
import pandas as pd
import seawater
import time
plt.rcParams.update({'font.size': 12})
font = {'family' : 'normal',
'weight' : 'normal',
'size'   : 12}

plt.rc('font', **font)

: 

## need wind speed at 10m for both UKESM and ERA5
for UKESM:
1) get daily u and v's from monthly files and save in monthly files, for all scenarios
2) regrid with cdo (u and v are on different grids and we need them on the same one to square)
3) concatenate files by year and get windspeed 

In [ ]:
fact = 1.1
fig, axs = plt.subplots(2,5, figsize=(12*fact, 8*fact), facecolor='w', edgecolor='k')
axs = axs.ravel()

: 

In [ ]:
demo = False
tdir = '/gpfs/home/mep22dku/scratch/SOZONE/windAnalyis/wspdComponents/'
if demo:
    w = xr.open_dataset('/gpfs/data/greenocean/software/resources/MetOffice/fixed_ozone/ssp126/u-be682_ssp126/be682_wind_y2070m10.nc')

    w2 = w.uwind10m.groupby('time_counter.day').mean()
    tdir = '/gpfs/home/mep22dku/scratch/SOZONE/windAnalyis/wspdComponents/'
    w3 = w2.to_dataset()
    w3.to_netcdf(f'{tdir}be682_wind_y2070m10_uwind10m_daily.nc')

    w4 = w.vwind10m.groupby('time_counter.day').mean()
    w5 = w4.to_dataset()
    w5.to_netcdf(f'{tdir}be682_wind_y2070m10_vwind10m_daily.nc')

uwind_orig = xr.open_dataset(f'{tdir}be682_wind_y2070m10_uwind10m_daily.nc')
uwind_rg = xr.open_dataset(f'{tdir}be682_wind_y2070m10_uwind10m_daily_rg.nc')
vwind_orig = xr.open_dataset(f'{tdir}be682_wind_y2070m10_vwind10m_daily.nc')
vwind_rg = xr.open_dataset(f'{tdir}be682_wind_y2070m10_vwind10m_daily_rg.nc')

fact = 1.1
fig, axs = plt.subplots(2,2, figsize=(12*fact, 8*fact), facecolor='w', edgecolor='k')
axs = axs.ravel()

axs[0].pcolormesh(vwind_orig.vwind10m[0,:,:])
axs[1].pcolormesh(vwind_rg.vwind10m[0,:,:])
axs[2].pcolormesh(uwind_orig.uwind10m[0,:,:])
axs[3].pcolormesh(uwind_rg.uwind10m[0,:,:])


: 

### ukesm windspeed from components

In [ ]:
tdir = '/gpfs/data/greenocean/software/resources/MetOffice/fixed_ozone/'
dir_1H = 'hist/u-bc370_hist/'
dir_2H = 'hist/u-cj198_hist_1950start1950ozone/'
dir_3H = 'hist/u-cj200_hist_1990start1990ozone/'

dir_1FA = 'ssp126/u-be682_ssp126/'
dir_1FB = 'ssp370/u-ce417_ssp370/'

dir_2FA = 'ssp126/u-cj880_ssp126_1950start1950ozone/'
dir_2FB = 'ssp370/u-cj881_ssp370_1950start1950ozone/'

dir_3FA = 'ssp126/u-cj484_ssp126_1990start1990ozone/'
dir_3FB = 'ssp370/u-cj504_ssp370_1990start1990ozone/'

name_1H = '1H: NatlOzone-historical'
name_2H = '2H: Ozone1950-historical'
name_3H = '3H: Ozone1990-historical'
name_1FA = '1FA: NatlOzone-SSP126'
name_1FB = '1FB: NatlOzone-SSP370'
name_2FA = '2FA: Ozone1950-SSP126'
name_2FB = '2FB: Ozone1950-SSP370'
name_3FA = '3FA: Ozone1990-SSP126'
name_3FB = '3FB: Ozone1990-SSP370'

col_1A = 'darksalmon'; col_1B = 'sienna'
col_2A = 'lightgreen'; col_2B = 'g'
col_3A = 'dodgerblue'; col_3B = 'mediumblue'

exp_1H = 'bc370'; exp_2H = 'cj198'; exp_3H = 'cj200'
exp_1FA = 'be682'; exp_1FB = 'ce417'
exp_2FA = 'cj880'; exp_2FB = 'cj881'
exp_3FA = 'cj484'; exp_3FB = 'cj504'

: 

In [ ]:
##1A

scen = '1A'
for y in range(1940,2101):
    if y < 2015:
        dir2 = dir_1H
    if y >= 2015:
        dir2 = dir_1FA
    fdir = f'{tdir}{dir2}'
    print(y)
    wx = glob.glob(f'{fdir}/*wind*y{y}*')
    wx2 = np.sort(wx)#wx.sort()
    mons = ['01','02','03','04','05','06','07','08','09','10','11','12']
    for i in range(0,12):
        if f'm{mons[i]}' not in wx2[i]:
            print(f'problem in {fdir}, {mons[i]}')
        else:
            pass
        w = xr.open_dataset(wx2[i])
        savedir = '/gpfs/data/greenocean/software/products/windsFromComponents/UKESM_monthly_atdayres/'
        savestr = f'UKESM_{scen}_y{y}m{mons[i]}'
        print(savestr)
        w2 = w.uwind10m.groupby('time_counter.day').mean()
        w3 = w2.to_dataset()
        w3['day'] = np.arange(i*30+1,(i+1)*30+1,1)
        w3.attrs = {'made in': '/gpfs/home/mep22dku/scratch/SOZONE/windAnalyis/wspdComponents/UKESMwspdFromComponents.ipynb'}
        w3.to_netcdf(f'{savedir}{savestr}_uwind10m_daily.nc')

        w4 = w.vwind10m.groupby('time_counter.day').mean()
        w5 = w4.to_dataset()
        w5['day'] = np.arange(i*30+1,(i+1)*30+1,1)
        w5.attrs = {'made in': '/gpfs/home/mep22dku/scratch/SOZONE/windAnalyis/wspdComponents/UKESMwspdFromComponents.ipynb'}
        w5.to_netcdf(f'{savedir}{savestr}_vwind10m_daily.nc')

##1A

scen = '1B'
print()
print(scen)
for y in range(1940,2101):
    if y < 2015:
        dir2 = dir_1H
    if y >= 2015:
        dir2 = dir_1FB
    fdir = f'{tdir}{dir2}'
    print(y)
    wx = glob.glob(f'{fdir}/*wind*y{y}*')
    wx2 = np.sort(wx)#wx.sort()
    mons = ['01','02','03','04','05','06','07','08','09','10','11','12']
    for i in range(0,12):
        if f'm{mons[i]}' not in wx2[i]:
            print(f'problem in {fdir}, {mons[i]}')
        else:
            pass
        w = xr.open_dataset(wx2[i])
        savedir = '/gpfs/data/greenocean/software/products/windsFromComponents/UKESM_monthly_atdayres/'
        savestr = f'UKESM_{scen}_y{y}m{mons[i]}'
        print(savestr)
        w2 = w.uwind10m.groupby('time_counter.day').mean()
        w3 = w2.to_dataset()
        w3['day'] = np.arange(i*30+1,(i+1)*30+1,1)
        w3.attrs = {'made in': '/gpfs/home/mep22dku/scratch/SOZONE/windAnalyis/wspdComponents/UKESMwspdFromComponents.ipynb'}
        w3.to_netcdf(f'{savedir}{savestr}_uwind10m_daily.nc')

        w4 = w.vwind10m.groupby('time_counter.day').mean()
        w5 = w4.to_dataset()
        w5['day'] = np.arange(i*30+1,(i+1)*30+1,1)
        w5.attrs = {'made in': '/gpfs/home/mep22dku/scratch/SOZONE/windAnalyis/wspdComponents/UKESMwspdFromComponents.ipynb'}
        w5.to_netcdf(f'{savedir}{savestr}_vwind10m_daily.nc')



: 

In [ ]:
##1A

scen = '2A'
for y in range(1940,2101):
    if y < 2015:
        dir2 = dir_2H
    if y >= 2015:
        dir2 = dir_2FA
    fdir = f'{tdir}{dir2}'
    print(y)
    wx = glob.glob(f'{fdir}/*wind*y{y}*')
    wx2 = np.sort(wx)#wx.sort()
    mons = ['01','02','03','04','05','06','07','08','09','10','11','12']
    for i in range(0,12):
        if f'm{mons[i]}' not in wx2[i]:
            print(f'problem in {fdir}, {mons[i]}')
        else:
            pass
        w = xr.open_dataset(wx2[i])
        savedir = '/gpfs/data/greenocean/software/products/windsFromComponents/UKESM_monthly_atdayres/'
        savestr = f'UKESM_{scen}_y{y}m{mons[i]}'
        print(savestr)
        w2 = w.uwind10m.groupby('time_counter.day').mean()
        w3 = w2.to_dataset()
        w3['day'] = np.arange(i*30+1,(i+1)*30+1,1)
        w3.attrs = {'made in': '/gpfs/home/mep22dku/scratch/SOZONE/windAnalyis/wspdComponents/UKESMwspdFromComponents.ipynb'}
        w3.to_netcdf(f'{savedir}{savestr}_uwind10m_daily.nc')

        w4 = w.vwind10m.groupby('time_counter.day').mean()
        w5 = w4.to_dataset()
        w5['day'] = np.arange(i*30+1,(i+1)*30+1,1)
        w5.attrs = {'made in': '/gpfs/home/mep22dku/scratch/SOZONE/windAnalyis/wspdComponents/UKESMwspdFromComponents.ipynb'}
        w5.to_netcdf(f'{savedir}{savestr}_vwind10m_daily.nc')

##1A

scen = '2B'
print()
print(scen)
for y in range(1940,2101):
    if y < 2015:
        dir2 = dir_2H
    if y >= 2015:
        dir2 = dir_2FB
    fdir = f'{tdir}{dir2}'
    print(y)
    wx = glob.glob(f'{fdir}/*wind*y{y}*')
    wx2 = np.sort(wx)#wx.sort()
    mons = ['01','02','03','04','05','06','07','08','09','10','11','12']
    for i in range(0,12):
        if f'm{mons[i]}' not in wx2[i]:
            print(f'problem in {fdir}, {mons[i]}')
        else:
            pass
        w = xr.open_dataset(wx2[i])
        savedir = '/gpfs/data/greenocean/software/products/windsFromComponents/UKESM_monthly_atdayres/'
        savestr = f'UKESM_{scen}_y{y}m{mons[i]}'
        print(savestr)
        w2 = w.uwind10m.groupby('time_counter.day').mean()
        w3 = w2.to_dataset()
        w3['day'] = np.arange(i*30+1,(i+1)*30+1,1)
        w3.attrs = {'made in': '/gpfs/home/mep22dku/scratch/SOZONE/windAnalyis/wspdComponents/UKESMwspdFromComponents.ipynb'}
        w3.to_netcdf(f'{savedir}{savestr}_uwind10m_daily.nc')

        w4 = w.vwind10m.groupby('time_counter.day').mean()
        w5 = w4.to_dataset()
        w5['day'] = np.arange(i*30+1,(i+1)*30+1,1)
        w5.attrs = {'made in': '/gpfs/home/mep22dku/scratch/SOZONE/windAnalyis/wspdComponents/UKESMwspdFromComponents.ipynb'}
        w5.to_netcdf(f'{savedir}{savestr}_vwind10m_daily.nc')




: 

In [ ]:
savedir = '/gpfs/data/greenocean/software/products/windsFromComponents/UKESM_monthly_atdayres/'
tw = [f'{savedir}UKESM_1A_y1940m01_uwind10m_daily.nc',f'{savedir}UKESM_1A_y1940m02_uwind10m_daily.nc',f'{savedir}UKESM_1A_y1940m03_uwind10m_daily.nc']
w = xr.open_mfdataset(tw)

: 

: 

: 